# Leikecase: Norske namn

## Bakgrunn

SSB har datagrunnlag for [10467: Fødte, etter jentenavn og guttenavn 1880 - 2022 ](https://www.ssb.no/statbank/table/10467/)
Tabellen inneheld namn over alle personar som er folkeregisterte i Noreg.
Dersom det er mindre enn fire personar med eit namn eitt gitt år, så vil det visast som ein tom verdi i dataen.

SSB har rota mase med "Unorske" teikn, og har difor erstatta
- È med E
- É med E
- Ô med O
- ' med blank
- Ä med Æ
- Ü med Y
- Ö med Ø

Eksempelvis består 5449 ANDRE av 
- 1831 André
- 72 Andrè
- 13 Andre'

Spørsmål som kan bli besvarte:
- Trender på ulike namn over tid
- Rangering av popularitet
- Mest populære namn per år, eller namn som er endra seg mest i popularitet 

## Kode

In [ ]:
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def get_ssb_data(p: str):
    url = "https://data.ssb.no/api/v0/no/table/10467/"
    with open(f"data/ssb_query_{p}.json") as f:
        payload = json.dumps(json.load(f))
    
    headers = {
      'Content-Type': 'application/json',
    }
    
    response = requests.request("POST", url, headers=headers, data=payload)
    return response.json()

In [ ]:
def json_to_records(j):
    navneliste = list(j["dimension"]["Fornavn"]["category"]["label"].values())
    årsliste = list(j["dimension"]["Tid"]["category"]["index"].keys())

    data = []
    i = 0
    for idx_navn, navn in enumerate(navneliste):
        for idx_år, år in enumerate(årsliste):
            data.append({
                "År": år,
                "Navn": navn,
                "Fødte": j["value"][i]
            })
            i += 1

    return data

### Hent navn gutter fra SSB

In [ ]:
g = get_ssb_data("gutter")
g_data = json_to_records(g)
df_g = pd.DataFrame.from_records(g_data)
df_g["Type"] = "G"
df_g.fillna(0, inplace=True)

### Hent navn for jenter fra SSB

In [ ]:
j = get_ssb_data("jenter")

In [ ]:
j_data = json_to_records(j)
df_j = pd.DataFrame.from_records(j_data)
df_j["Type"] = "J"
df_j.fillna(0, inplace=True)

In [ ]:
df = pd.concat([df_j, df_g])
df["År"] = df["År"].astype(pd.Int64Dtype())

In [ ]:
total_births = df.pivot_table("Fødte", index="År", columns="Type", aggfunc="sum")

In [ ]:
total_births.plot()

In [ ]:
df = df[df["År"] >= 1945]

In [ ]:
def add_andel(group):
    group["andel"] = group["Fødte"] / group["Fødte"].sum()
    return group

In [ ]:
df = df.groupby(["År", "Type"], group_keys=False).apply(add_andel)

In [ ]:
df[df["Navn"] == "Aagot"]

In [ ]:
df.groupby(["År", "Type"])["andel"].sum()

In [ ]:
def get_top100(group):
    return group.sort_values("Fødte", ascending=False)[:100]

In [ ]:
top100 = df.groupby(["År", "Type"]).apply(get_top100).reset_index(drop=True)

In [ ]:
top100.head()

In [ ]:
total_births = top100.pivot_table("Fødte", index="År", columns="Navn", aggfunc="sum").fillna(0)
total_andel = top100.pivot_table("andel", index="År", columns="Navn", aggfunc="sum").fillna(0)

In [ ]:
total_births.info()

In [ ]:
total_births.head()

In [ ]:
subset = total_births[["Silje", "Camilla", "Espen"]]

In [ ]:
subset.plot() 

In [ ]:
t = top100.pivot_table("andel", index="År", columns="Type", aggfunc="sum")

In [ ]:
t

In [ ]:
t.plot()

Kor mange distinkte namn i topp 50%?

In [ ]:
boys = df[df["Type"] == "G"]

In [ ]:
df_1950_boys = boys[boys["År"] == 1950]

In [ ]:
andel_1950_boys = df_1950_boys["andel"].sort_values(ascending=False).cumsum()

In [ ]:
andel_1950_boys.searchsorted(0.5) + 1

In [ ]:
df_2022_boys = boys[boys["År"] == 2022]

In [ ]:
andel_2022_boys = df_2022_boys["andel"].sort_values(ascending=False).cumsum()

In [ ]:
andel_2022_boys.searchsorted(0.5) + 1

In [ ]:
def get_quantile_count(group, q=0.5):
    group = group.sort_values("andel", ascending=False)
    return group["andel"].cumsum().searchsorted(q) + 1

In [ ]:
mangfold = df.groupby(["År", "Type"]).apply(get_quantile_count)

In [ ]:
mangfold = mangfold.unstack()

In [ ]:
mangfold.plot()

## Endring over tid

In [ ]:
def add_delta(group):
    group["delta"] = group["Fødte"] - group["Fødte"].shift()
    return group

In [ ]:
df_delta = df.groupby(["Navn"], group_keys=False).apply(add_delta)

In [ ]:
df_delta[df_delta["Navn"] == "Aage"].plot(x="År", y="delta", kind="bar")

## Endring i første bokstav

In [ ]:
first_letters = df["Navn"].map(lambda n: n[0])
first_letters.name = "first_letter"
first_letters

In [ ]:
table = df.pivot_table("Fødte", index=first_letters, columns=["Type", "År"], aggfunc="sum")

In [ ]:
subtable = table.reindex(columns=[1980, 2000, 2020], level="År")

In [ ]:
letter_prop = subtable / subtable.sum()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 8))
letter_prop["G"].plot(kind="bar", rot=0, ax=axes[0], title="G")
letter_prop["J"].plot(kind="bar", rot=0, ax=axes[1], title="J",
                      legend=False)